# Neural & Behavioral Modeling - Week 4 (Exercises)
by Your Name (Your Email)

In [2]:
%config IPCompleter.greedy=True 
%matplotlib inline
from numpy import *
from matplotlib.pyplot import *
from IPython.display import *

## 1 Replicate exisiting findings/models 
Based on your personal interest, please choose and replicate ONE of the following studies:

1. <a href="http://ccl.northwestern.edu/netlogo/models/FlockingVeeFormations">Flocking Vee Formations</a> in NetLogo's [Sample Models::Biology] 

2. <a href="http://ccl.northwestern.edu/netlogo/models/WolfSheepPredation">Wolf-Sheep Predation</a> in NetLogo's [Sample Models::Biology] 

3. <a href="https://www.wetalk.tw/thread-36278-1-1.html">MIT Matching Game</a>

4. Gray, K., Rand, D. G., Ert, E., Lewis, K., Hershman, S., & Norton, M. I. (2014). <a href="http://www.mpmlab.org/groups/">The emergence of “us and them” in 80 lines of code: Modeling group genesis in homogeneous populations</a>. Psychological Science, 25(4), 982-990.

5. Luhmann, C. C., & Rajaram, S. (2015). <a href="http://journals.sagepub.com/doi/abs/10.1177/0956797615605798">Memory transmission in small groups and large networks: An agent-based model</a>. Psychological Science, 26(12), 1909-1917.

In [1]:
# Write your codes here

In [3]:
# Model parameters:
Nboids=30 
world=[0,500,0,500] # world boundaries
speed=10 
repulsion_dis=10   # repulsion distance
alignment_dis=80   # alignment distance
attraction_dis=100 # attraction distance

obstruct_dis=40
scopedegree = 22.5

# Supporting functions:
class Boid:
    
    def __init__(self,world):
        world_size=max(world)
        self.position=world_size*random.rand(2) 
        temp_dir=random.rand(2)
        self.direction=temp_dir/linalg.norm(temp_dir)
        
    def move(self,world,boids,distance,obstruct):
        
        # Repulsion to group position is the top priority:
        repulsion_group=[boids[j] for j in range(Nboids) if distance[j]>0 and distance[j]<=repulsion_dis \
                         and obstruct[j]<=50]
        Nrepulsion=len(repulsion_group)
        if(Nrepulsion>0):  
            
            group_position=zeros(2)
            for r in repulsion_group:
                group_position+=r.position
            group_position/=Nrepulsion
            
            # set rather than revise the heading direction:
            self.direction=self.position-group_position
                        
        else:
            
            # Alignment to group direction:
            alignment_group=[boids[j] for j in range(Nboids) if distance[j]>repulsion_dis and distance[j]<=alignment_dis \
                            and obstruct[j]<=50]
            Nalignment=len(alignment_group)
            if(Nalignment>0):  
                group_direction=zeros(2)
                for a in alignment_group:
                    group_direction+=a.direction # addition of unit vectors
                self.direction+=group_direction # revise the original direction
                
            # Attraction to group position:
            attraction_group=[boids[j] for j in range(Nboids) if distance[j]>alignment_dis and distance[j]<=attraction_dis\
                             and obstruct[j]<=50]
            Nattraction=len(attraction_group)
            if(Nattraction>0):  
                group_position=zeros(2)
                for a in attraction_group:
                    group_position+=a.position
                group_position/=Nattraction
                catch_direction=(group_position-self.position)
                self.direction+=catch_direction # revise the original direction
            #--------------------------------
            flockingvee_group = [boids[j] for j in range(Nboids) if distance[j]<obstruct_dis and obstruct[j]<=scopedegree]
            
            Nflockingvee=len(flockingvee_group)
            if(Nflockingvee>0): 
                group_direction=zeros(2)
                for a in alignment_group:
                    group_direction+=a.direction # addition of unit vectors
                self.direction+=group_direction # revise the original direction            
            #--------------------------------
        # For all cases (including the case of no neighbors at all):
        self.direction/=linalg.norm(self.direction) # make it a unit vector
        self.position=around(self.position+self.direction*speed)
        self.position=mod(self.position,max(world)) # cyclic boundary

def plot_world(world,boids):
    clf() # clear previous figure
    for b in boids:
        plot(b.position[0],b.position[1],'r^')
    axis(world)
    display(gcf()); clear_output(wait=True) # to allow dynamic plots
    
# Initialization:
boids=[Boid(world) for i in range(Nboids)] # assign each boid to a position

for t in range(1000):
    
    # Calculate all pairwise distances before anyone moves:
    distance=zeros([Nboids,Nboids])
    #--------------------------------
    obstruct=zeros([Nboids,Nboids])
    #--------------------------------
    for i in range(Nboids):
        for j in range(i+1,Nboids):
            distance[i,j]=linalg.norm(boids[i].position-boids[j].position)
    #--------------------------------
            v0 = boids[i].direction
            v1 = boids[j].direction
            angle = math.atan2(linalg.det([v0,v1]),dot(v0,v1))
            obstruct[i,j] = degrees(angle)        
    #--------------------------------        
    # Move according to the three rules:       
    for i in range(Nboids):
        boids[i].move(world,boids,distance[i,:],obstruct[i,:])
        
    plot_world(world,boids)

KeyboardInterrupt: 